In [2]:
from edward.models import MultivariateNormalTriL, Normal, Poisson
from edward.util import rbf
import numpy as np
import pandas as pd
import edward as ed
import tensorflow as tf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import GPy
GPy.plotting.change_plotting_library('plotly')
from IPython.display import display
init_notebook_mode(connected=True)

In [3]:
counts = pd.read_csv('data/zip_counts.csv')
counts = counts[(counts['LAT'] > 25) & (counts['LAT'] < 50) & (counts['LNG'] > -125) & (counts['LNG'] < -67) &(counts['population'] > 0)]

In [4]:
step = 0.25
to_bin = lambda x: np.floor(x / step) * step
counts["latbin"] = counts.LAT.map(to_bin)
counts["lonbin"] = counts.LNG.map(to_bin)
grouped = pd.DataFrame(counts.groupby(['latbin', 'lonbin', 'committee_name'], as_index = False)['count', 'population'].sum())

In [5]:
clinton = grouped[grouped['committee_name'] == 'HILLARY FOR AMERICA']
iplot([go.Scatter3d(x = clinton['lonbin'], y = clinton['latbin'], z=clinton['count'], mode = 'markers', marker=dict(size = 2,))])

## Edward

In [6]:
from edward.models import MultivariateNormalTriL, Normal, Poisson
from edward.util import rbf

In [7]:
N = clinton.shape[0]
D = 2

x_ph = tf.placeholder(tf.float32, (N,D))  # inputs to Gaussian Process
K = rbf(x_ph, lengthscale=2.5, variance = 0.1) + tf.diag(tf.ones(N)*1e-3)
f = MultivariateNormalTriL(loc=tf.ones((N))*np.log(clinton['count'].mean()), scale_tril=tf.contrib.distributions.matrix_diag_transform(K))
y = Poisson(rate=tf.exp(f))

In [8]:
qf = Normal(loc=tf.Variable(tf.random_normal([N])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

inference = ed.KLqp({f: qf}, data={x_ph: np.array(clinton[['latbin','lonbin']]), y: np.array(clinton['count'])})
inference.run(n_iter=1000)

1000/1000 [100%] ██████████████████████████████ Elapsed: 371s | Loss: 299547.312

In [374]:
pred = qf.mean().eval()
iplot([go.Scatter3d(x = X[:,1], y = X[:,0], z=pred, mode = 'markers', marker=dict(
        size = 2,
    ))])